## Details of the processing and manipulation of data in Python
Before we can do data analysis, we need to clean the data. 
Data cleansing is a prerequisite for data analysis and it is also known as pre-processing. 

In [1]:
import os

import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None

# Data Cleaning
## Import the data
When importing data, we first need to find a list of all the data files and make sure that the path we find matches where all the files are running. 
Then merge all the separate data files into a single CSV. 
For example, we combine separate data into energy_CSVs and temperature_CSVs. 
This ensures that our cleanup and analysis are synchronized. 
Merge the data than the separation of data cleaning or analyze the data more convenient, clearer.
Here we need to pay attention to, this code may be through the use of glob rather than os.listdir to simplify.

In [2]:
energy_locations = os.listdir("../Data/Energy")
temperature_locations = os.listdir("../Data/Temperature")

energy_CSVs = [pd.read_csv("../Data/Energy/" + location) for location in energy_locations]
temperature_CSVs = [pd.read_csv("../Data/Temperature/" + location) for location in temperature_locations if "Data" in location]

E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,22,24,26,28,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,28,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,16,22,24,26,28,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
energy_data = pd.concat(energy_CSVs, ignore_index=True)
temperature_data = pd.concat(temperature_CSVs, ignore_index=True)

# Note some excess data is shaved off here for simplicity
stations_data = pd.read_csv("../Data/Temperature/HM01X_StnDet_999999999743964.txt", skiprows=5, header=None, usecols=range(1, 21), names=["StationNumber", "RainfulDistrict", "Station", "StationOpenDate", "StationCloseDate", "Latitude", "Longitude", "LocationMethod", "State", "StationHeight", "BarometerHeight", "WMO", "FirstYear", "LastYear", "CompletionPercent", "YQualityPercent", "NQualityPercent", "WQualityPercent", "SQualityPercent", "IQualityPercent"])

## Column standardising
Next, we have to do is to standardize the column. 
In the raw data, they might have the following errors, such as misprints in names, the same properties with different names, identical separate classes, and case inconsistencies. 
First, we renamed all the columns in a standardized way to make the data clearer and more complete. 
We want to the simplest form of writing, there is no space and the first letter in alphabetical order. 
This standardization occurs when station information is imported. 
This kind of cleaning will improve the efficiency of our model and give better results.

In [4]:
energy_data.dtypes
temperature_data.dtypes
stations_data.dtypes

REGION             object
SETTLEMENTDATE     object
TOTALDEMAND       float64
RRP               float64
PERIODTYPE         object
dtype: object

hm                                                         object
Station Number                                              int64
Year Month Day Hour Minutes in YYYY                         int64
MM                                                          int64
DD                                                          int64
HH24                                                        int64
MI format in Local time                                     int64
Year Month Day Hour Minutes in YYYY.1                       int64
MM.1                                                        int64
DD.1                                                        int64
HH24.1                                                      int64
MI format in Local standard time                            int64
Precipitation since 9am local time in mm                   object
Quality of precipitation since 9am local time              object
Air Temperature in degrees C                               object
Quality of

StationNumber          int64
RainfulDistrict       object
Station               object
StationOpenDate       object
StationCloseDate      object
Latitude             float64
Longitude            float64
LocationMethod        object
State                 object
StationHeight        float64
BarometerHeight      float64
WMO                    int64
FirstYear              int64
LastYear               int64
CompletionPercent      int64
YQualityPercent        int64
NQualityPercent        int64
WQualityPercent        int64
SQualityPercent        int64
IQualityPercent        int64
dtype: object

In [5]:
energy_data.columns = ["Region", "Date", "TotalDemand", "RRP", "PeriodType"]
temperature_data.columns = [
    "HM", "StationNumber", "Year1", "Month1", "Day1", "Hour1", "Minute1", "Year", "Month", "Day", "Hour", "Minute", "Precipitation", "PrecipitationQuality",
    "AirTemperature", "AirTemperatureQuality", "WetBulbTemperature", "WetBulbTemperatureQuality", "DewTemperature", "DewTemperatureQuality", "RelativeHumidity",
    "RelativeHumidityQuality", "WindSpeed", "WindSpeedQuality", "WindDirection", "WindDirectionQuality", "WindgustSpeed", "WindgustSpeedQuality", "SeaPressure",
    "SeaPressureQuality", "StationPressure", "StationPressureQuality", "AWSFlag", "#"
]

## Remove any easily found unnecessary data
Then we need to remove any unnecessary data that is easy to find. 
During data collection, repeated or redundant observations are the most common. 
Duplication of data changes efficiency to a large extent, and may increase the right or wrong side, resulting in disloyal results. 
The redundant data results have no effect on our analysis and can be deleted directly. 
Delete any data that is obviously duplicate or that always has the same value or that is meaningless for analysis. 
A simple look at the data gives you an overview of which data might be easily deleted! 
For example, HM, #, Period type.
They have no practical effect on our analysis, so we can delete them.

In [6]:
energy_data
temperature_data
stations_data

,Region,Date,TotalDemand,RRP,PeriodType
0,NSW1,2000/01/01 00:30,6763.57000,15.64,TRADE
1,NSW1,2000/01/01 01:00,6386.10167,14.06,TRADE
2,NSW1,2000/01/01 01:30,5990.79500,14.30,TRADE
3,NSW1,2000/01/01 02:00,5655.97667,14.28,TRADE
4,NSW1,2000/01/01 02:30,5283.83667,14.17,TRADE
...,...,...,...,...,...
1658960,VIC1,2019/12/31 22:00:00,4129.96000,52.91,TRADE
1658961,VIC1,2019/12/31 22:30:00,4083.66000,58.66,TRADE
1658962,VIC1,2019/12/31 23:00:00,4104.95000,54.36,TRADE
1658963,VIC1,2019/12/31 23:30:00,4325.88000,66.87,TRADE


,HM,StationNumber,Year1,Month1,Day1,Hour1,Minute1,Year,Month,Day,Hour,Minute,Precipitation,PrecipitationQuality,AirTemperature,AirTemperatureQuality,WetBulbTemperature,WetBulbTemperatureQuality,DewTemperature,DewTemperatureQuality,RelativeHumidity,RelativeHumidityQuality,WindSpeed,WindSpeedQuality,WindDirection,WindDirectionQuality,WindgustSpeed,WindgustSpeedQuality,SeaPressure,SeaPressureQuality,StationPressure,StationPressureQuality,AWSFlag,#
0,hm,23090,2000,1,1,1,0,2000,1,1,0,0,0,N,13.1,N,10.0,N,6.6,N,65,N,14.8,N,190,N,24.1,N,1018.9,N,1012.8,N,,#
1,hm,23090,2000,1,1,1,30,2000,1,1,0,30,0,N,13.2,N,10.1,N,6.8,N,65,N,9.4,N,200,N,16.6,N,1018.7,N,1012.6,N,,#
2,hm,23090,2000,1,1,2,0,2000,1,1,1,0,0,N,13.4,N,10.1,N,6.5,N,63,N,13,N,180,N,20.5,N,1018.5,N,1012.4,N,,#
3,hm,23090,2000,1,1,2,30,2000,1,1,1,30,0,N,13.2,N,10.0,N,6.5,N,64,N,9.4,N,170,N,18.4,N,1018.3,N,1012.2,N,,#
4,hm,23090,2000,1,1,3,0,2000,1,1,2,0,0,N,13.0,N,9.7,N,6.0,N,62,N,11.2,N,150,N,20.5,N,1018.3,N,1012.2,N,,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826239,hm,94029,2020,1,20,8,0,2020,1,20,7,0,0.0,N,14.9,N,12.5,N,10.3,N,74,N,14.8,N,130,N,20.5,N,1011.4,N,1005.3,N,1,#
1826240,hm,94029,2020,1,20,8,30,2020,1,20,7,30,0.0,N,14.9,N,12.4,N,10.1,N,73,N,13,N,140,N,18.4,N,1011.5,N,1005.4,N,1,#
1826241,hm,94029,2020,1,20,9,0,2020,1,20,8,0,0.0,N,14.9,N,12.4,N,10.1,N,73,N,16.6,N,150,N,24.1,N,1011.7,N,1005.6,N,1,#
1826242,hm,94029,2020,1,20,9,30,2020,1,20,8,30,0.0,N,15.1,N,12.4,N,9.9,N,71,N,16.6,N,150,N,22.3,N,1011.6,N,1005.5,N,1,#


,StationNumber,RainfulDistrict,Station,StationOpenDate,StationCloseDate,Latitude,Longitude,LocationMethod,State,StationHeight,BarometerHeight,WMO,FirstYear,LastYear,CompletionPercent,YQualityPercent,NQualityPercent,WQualityPercent,SQualityPercent,IQualityPercent
0,86338,86,MELBOURNE (OLYMPIC PARK),05/2013,,-37.8255,144.9816,SURVEY,VIC,7.5,7.5,95936,2013,2020,103,0,100,0,0,0
1,86071,86,MELBOURNE REGIONAL OFFICE,01/1908,01/2015,-37.8075,144.9700,GPS,VIC,31.2,32.2,94868,2000,2015,99,0,100,0,0,0
2,23090,23A,ADELAIDE (KENT TOWN),01/1977,,-34.9211,138.6216,GPS,SA,48.0,51.0,94675,2000,2020,101,0,100,0,0,0
3,66062,66,SYDNEY (OBSERVATORY HILL),01/1858,,-33.8607,151.2050,GPS,NSW,39.0,40.2,94768,2000,2020,99,0,100,0,0,0
4,94029,94,HOBART (ELLERSLIE ROAD),01/1882,,-42.8897,147.3278,GPS,TAS,50.5,51.4,94970,2000,2020,110,0,100,0,0,0
5,40913,40,BRISBANE,12/1999,,-27.4808,153.0389,GPS,QLD,8.1,8.3,94576,2000,2020,99,0,100,0,0,0


In [7]:
def remove_non_uniques(dataframe: pd.DataFrame, filter = []):
    remove = [name for name, series in dataframe.items() if len(series.unique()) <= 2 and not name in filter]
    dataframe.drop(remove, axis=1, inplace=True)
    return remove

print("Removed:")
remove_non_uniques(energy_data)
remove_non_uniques(temperature_data)
remove_non_uniques(stations_data, "LocationMethod")

# Manually remove extra columns
stations_data.drop("StationOpenDate", axis=1, inplace=True)

Removed:


['PeriodType']

['HM',
 'PrecipitationQuality',
 'AirTemperatureQuality',
 'WetBulbTemperatureQuality',
 'DewTemperatureQuality',
 'RelativeHumidityQuality',
 'WindSpeedQuality',
 'WindDirectionQuality',
 'WindgustSpeedQuality',
 'SeaPressureQuality',
 'StationPressureQuality',
 '#']

['StationCloseDate',
 'FirstYear',
 'LastYear',
 'YQualityPercent',
 'NQualityPercent',
 'WQualityPercent',
 'SQualityPercent',
 'IQualityPercent']

In [8]:
energy_data.columns
temperature_data.columns
stations_data.columns

Index(['Region', 'Date', 'TotalDemand', 'RRP'], dtype='object')

Index(['StationNumber', 'Year1', 'Month1', 'Day1', 'Hour1', 'Minute1', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Precipitation', 'AirTemperature',
       'WetBulbTemperature', 'DewTemperature', 'RelativeHumidity', 'WindSpeed',
       'WindDirection', 'WindgustSpeed', 'SeaPressure', 'StationPressure',
       'AWSFlag'],
      dtype='object')

Index(['StationNumber', 'RainfulDistrict', 'Station', 'Latitude', 'Longitude',
       'LocationMethod', 'State', 'StationHeight', 'BarometerHeight', 'WMO',
       'CompletionPercent'],
      dtype='object')

## Deal with multiple dates
There are multiple dates in the temperature data and we need to process them. 
First, we need to find the difference between the dates. 
Then make sure that all the dates are combined into one column. 
Comments provided by the regulation, the date of the first group is local, the second group is the local standard time. 
Although in general is the same, but contains a daylight saving time, local time, the clock to move or moved forward one hour after the meeting. 
For the sake of simplicity, we used the local standard time. 
We set the previous local standard time column names to no 1 and the other column names to 1 to avoid renaming.

In [9]:
# Remove extra dates
temperature_data.drop(["Year1", "Month1", "Day1", "Hour1", "Minute1"], axis=1, inplace=True)

# Reformat dates into Pandas' datatime64 objects
# Replacing old format
temperature_data["Date"] = pd.to_datetime(temperature_data[["Year", "Month", "Day", "Hour", "Minute"]])
energy_data["Date"] = pd.to_datetime(energy_data["Date"])

temperature_data.drop(["Year", "Month", "Day", "Hour", "Minute"], axis=1, inplace=True)

## Check for duplicated rows
During preprocessing, we also need to check for duplicate rows. 
Because no specific selection is provided, this procedure applies to all columns. 
In order to check which rows are deleted, you can use Panda's duplication feature in advance. 
For the sake of brevity, this function is not included in the scope of analysis, in the process of checking we found no duplicate rows.

In [10]:
energy_data.drop_duplicates(inplace=True)
temperature_data.drop_duplicates(inplace=True)
stations_data.drop_duplicates(inplace=True)

## Converting datatypes
The next step is to convert the data types. 
To ensure that we can create visual, graphical and mathematical, statistical models, we need to use digital data types instead of objects! For AWS Flag, we assume that null values are recorded manually. 
For AWS Flag, null values are assumed to be manual because this is the worst result.


In [11]:
def to_object_columns(lambda_function):
    string_columns = temperature_data.select_dtypes("object").columns
    temperature_data[string_columns] = temperature_data[string_columns].apply(lambda_function)

In [12]:
to_object_columns(lambda column: column.str.strip())

In [13]:
temperature_data["AWSFlag"] = temperature_data["AWSFlag"].replace("", 0).astype("category")
temperature_data["AWSFlag"].fillna(0, inplace=True)
temperature_data["RelativeHumidity"] = temperature_data["RelativeHumidity"].replace("###", np.NaN)

In [14]:
to_object_columns(lambda column: pd.to_numeric(column))

In [15]:
temperature_data.dtypes

StationNumber                  int64
Precipitation                float64
AirTemperature               float64
WetBulbTemperature           float64
DewTemperature               float64
RelativeHumidity             float64
WindSpeed                    float64
WindDirection                float64
WindgustSpeed                float64
SeaPressure                  float64
StationPressure              float64
AWSFlag                     category
Date                  datetime64[ns]
dtype: object

## Finding missing data
One of the seemingly intractable problems is the "lack of data". 
For clarity we cannot simply ignore missing values in the dataset. 
For very practical reasons, we have to deal with the missing data in some way, because most application algorithms do not accept data sets with missing values. 
To simplify the output, columns with no null values are ignored. 
The energy data has no null value. 
The null array shows this, and the temperature data shows the number of empty cells per column.
We must manually save the cleaned data so that we can visualize the missing data. 
As we can see from the chart, a large chunk of wind data is missing, so our analysis is useless. 
Therefore, we deleted the data. 
The figure also shows that it should be safe to interpolate missing values.

In [16]:
def get_null_counts(dataframe: pd.DataFrame):
    return dataframe.isnull().mean()[dataframe.isnull().mean() > 0]

In [17]:
get_null_counts(energy_data)
get_null_counts(temperature_data)

Series([], dtype: float64)

Precipitation         0.229916
AirTemperature        0.444437
WetBulbTemperature    0.011324
DewTemperature        0.375311
RelativeHumidity      0.375312
WindSpeed             0.532966
WindDirection         0.432305
WindgustSpeed         0.403183
SeaPressure           0.137730
StationPressure       0.011135
dtype: float64

In [18]:
pd.to_pickle([energy_data, temperature_data, stations_data], "../Data/CleanedData.pickle")

In [19]:
temperature_data.drop(["WindSpeed", "WindgustSpeed", "WindDirection"], axis=1, inplace=True)

# Note that using inplace currently throws an error
# So interpolated columns must be manually overridden
missing_columns = list(get_null_counts(temperature_data).keys())
temperature_data[missing_columns] = temperature_data[missing_columns].interpolate(method="linear")

## Grouping data
Next， we need to group the data. 
We put our energy and temperature data are recorded in a particular area. 
The naming of both needs to be standardized so that they can be easily and smoothly used as the Pandas group! 
We found that there was more than one weather station in the state of Victoria for some time periods and that the two weather stations here were effectively combined into one. 
While this may cause a slight bias in the data set, the current assumption is that the impact is small.


In [20]:
energy_data["Region"].unique()
temperature_data["StationNumber"].unique()

array(['NSW1', 'QLD1', 'SA1', 'VIC1', 'TAS1'], dtype=object)

array([23090, 40913, 66062, 86071, 86338, 94029], dtype=int64)

In [21]:
region_remove_number_map = {"SA1": "SA", "QLD1": "QLD", "NSW1": "NSW", "VIC1": "VIC", "TAS1": "TAS"}
station_to_region_map = {23090: "SA", 40913: "QLD", 66062: "NSW", 86071: "VIC", 94029: "TAS", 86338: "VIC"}

temperature_data["Region"] = temperature_data["StationNumber"].map(station_to_region_map)
energy_data["Region"] = energy_data["Region"].map(region_remove_number_map)

temperature_data.drop("StationNumber", axis=1, inplace=True)

## Combining energy and temperature data
We also need to combine energy and temperature data. 
In order to combine the two data frame, we must first ensure that the two data sets are sorted by date, and then we shall be carried out in accordance with the closest to the value of the merger. 
We must manually specify the column values to be grouped by parameter, and we must specify a reasonable tolerance.
Our data is 30 minutes record for the unit, so the tolerance is reasonable! 
If there are no null values or very few null values, that means the process is happening. 
No or very few null values indicate that the process has been relatively successful. 
We can do this because the frequency at which the data is recorded is quite consistent across the energy and temperature data sets! 
If not, then we can minimize the frequency of the data. 
If this is not the case, you have to down sampling on a data set. 
We remove any remaining null values, because their number is relatively small.


In [22]:
energy_data.sort_values(by="Date", inplace=True)
temperature_data.sort_values(by="Date", inplace=True)

data = pd.merge_asof(energy_data, temperature_data, on="Date", by="Region", tolerance=pd.Timedelta("30 min"))
data.set_index("Date", inplace=True)

In [23]:
get_null_counts(data)
data.dropna(inplace=True)

Precipitation         0.011302
AirTemperature        0.001634
WetBulbTemperature    0.001634
DewTemperature        0.001634
RelativeHumidity      0.001634
SeaPressure           0.001634
StationPressure       0.001634
AWSFlag               0.001634
dtype: float64

In [24]:
data

,Region,TotalDemand,RRP,Precipitation,AirTemperature,WetBulbTemperature,DewTemperature,RelativeHumidity,SeaPressure,StationPressure,AWSFlag
Date,,,,,,,,,,,
2000-01-01 00:30:00,NSW,6763.57000,15.64,0.000000,17.841568,14.1,11.821183,68.136687,1011.100000,1006.4,0
2000-01-01 00:30:00,QLD,3905.56833,39.40,0.000000,18.594473,19.2,14.600415,93.998779,1007.200000,1006.2,0
2000-01-01 00:30:00,VIC,4419.03667,9.97,0.000000,13.700000,10.5,7.200000,65.000000,1017.000000,1013.1,0
2000-01-01 01:00:00,VIC,4312.54000,12.58,0.000000,13.600000,10.3,6.700000,63.000000,1016.800000,1012.9,0
2000-01-01 01:00:00,NSW,6386.10167,14.06,0.000000,17.841429,13.7,11.821143,68.137140,1011.000000,1006.3,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 00:00:00,TAS,1006.70000,93.88,0.000000,17.900000,9.5,9.100000,56.000000,1016.300000,1004.6,0
2020-01-01 00:00:00,QLD,6218.39000,52.15,0.000000,17.970884,21.6,11.858802,67.713472,1011.440920,1013.8,0
2020-01-01 00:00:00,SA,1474.11000,85.95,0.000000,18.469005,12.3,8.700000,61.000000,1008.290509,1010.6,0


## Saving final model
The last step in our preprocessing is saving final model.

In [25]:
pd.to_pickle(data, "../Data/Data.pickle")